<a href="https://colab.research.google.com/github/KsuPav/diploma/blob/main/%D0%A7%D0%B8%D1%81%D1%82%D0%BE%D0%B2%D0%B8%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

import xgboost as xgb
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, classification_report, multilabel_confusion_matrix,  recall_score, precision_score, f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split



In [ ]:
# Импортируем базу
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

import io
data = pd.read_csv(io.StringIO(uploaded['data.csv'].decode('utf-8')))

In [ ]:
# Проверка типов
print(data.dtypes)


In [ ]:
# Убираем ненужные признаки
data = data.drop(['названиерегиона','Unnamed: 0', 'Код_ученика'], axis=1)


In [ ]:
# Кодирование
encoder = LabelEncoder()
data['ЛогинОО'] = encoder.fit_transform(data1['ЛогинОО'])
print(data.dtypes)

In [ ]:
# Разделение параметров и меток
features = data.drop(['Балл', 'Отметка'], axis=1)
labels = data['Отметка']


# train-test (85% - обучение, 15% - валидация)
x_train, x_test, y_train, y_test = train_test_split(features, labels,  train_size=0.85)

In [ ]:
# Преобразование в DMatrix
d_train = xgb.DMatrix(x_train, y_train)
d_test = xgb.DMatrix(x_test, y_test)

In [ ]:
# Настройка параметров
params_XGB = {
    'tree_method': 'approx', # 'auto' не подходит, слижком большой датасет
    'booster': 'gbtree',
    'max_depth': 5, 
    'learning_rate': 0.1,
    'sample_type': 'uniform',
    'normalize_type': 'tree',
    'objective': 'reg:squarederror', # регрессия с квадратом потерь
    'rate_drop': 0.1,
    'n_estimators': 500
}

In [ ]:
xgb_clf = xgb.train(params_XGB, d_train)

In [ ]:
# Визуализация важности параметров
plt.figure(figsize=(16, 12))
xgb.plot_importance(xgb_clf)
plt.show()

In [ ]:
# Прогноз
preds = xgb_clf.predict(d_test)

In [ ]:
preds_round = preds.round().astype(int)

In [ ]:
# Процент ошибок в прогнозировании
print(round((y_test != preds_round).sum() / y_test.size * 100), '%')

In [ ]:
print(accuracy_score(y_test, preds_round))

In [ ]:
print(np.round(mean_squared_error(y_test, preds_round)))

In [ ]:
# DMatrix, состоящая из всех обучающих данных
dmatrix_data_1 = xgb.DMatrix(features, labels_1)
dmatrix_data_2 = xgb.DMatrix(features, labels_2)

In [ ]:
# Параметры
params = {
    'objective': 'reg:squarederror', # регрессия с квадратом потерь
    'colsample_bytree': 0.3,
    'learning_rate': 0.1,
    'max_depth': 5,
}

In [ ]:
# Кросс-валидация
cross_val = xgb.cv(
    params=params,
    dtrain=dmatrix_data_1, 
    nfold=3,
    num_boost_round=50,
    early_stopping_rounds=10,
    metrics='error',
    as_pandas=True,
    seed=42)

print(cross_val.head())